# Get raw data (order_processed.pkl)


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
from datetime import datetime, timedelta
import pickle
import FinanceDataReader as fdr
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
transaction = pd.read_pickle('order_processed.pkl')
transaction = transaction[['account_id', 'code', 'quantity', 'price', 'side', 'created_at']]
# exception
transaction = transaction[~transaction['created_at'].astype(str).str.startswith('2022-03-09')]
transaction

,account_id,code,quantity,price,side,created_at
0,8a9fa71f8b77775af51dfb5db302f272,290270,200,5960.0,bid,2021-03-15 08:40:18.983000+09:00
2,8a9fa71f8b77775af51dfb5db302f272,043200,3000,1480.0,bid,2021-03-15 08:41:05.676000+09:00
3,4e1334d8f5e21bb282aff7ee073efbcb,011785,17,91600.0,bid,2021-03-15 08:41:41.135000+09:00
4,8a9fa71f8b77775af51dfb5db302f272,262260,100,39550.0,bid,2021-03-15 08:41:41.807000+09:00
5,4e1334d8f5e21bb282aff7ee073efbcb,064260,501,9000.0,bid,2021-03-15 08:42:06.243000+09:00
...,...,...,...,...,...,...
962811,93a9826e66eb135e677d7da9a5bbc923,032500,3000,19250.0,bid,2023-07-03 15:23:56.291000+09:00
962812,93a9826e66eb135e677d7da9a5bbc923,032500,3000,19250.0,bid,2023-07-03 15:23:58.521000+09:00
962815,93a9826e66eb135e677d7da9a5bbc923,032500,1985,14720.0,bid,2023-07-03 15:24:29.408000+09:00
962816,80dcbc851e5e8836a1e9b40efbe42a91,089590,8892,15070.0,ask,2023-07-03 15:25:17.360000+09:00


In [3]:
"""
Rolling Window
create period folder
"""

PERIOD = 1
Path(f'period_{PERIOD}').mkdir(parents=True, exist_ok=True)
save_path = f'period_{PERIOD}/'

def divide_periods(start_date, num_periods):
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    
    period_duration = timedelta(days=9*30.5)  # Assuming 30 days per month
    
    periods = []
    for i in range(num_periods):
        end_date = start_date + period_duration - timedelta(days=1)
        end_date = end_date.replace(day=14)
        periods.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
        start_date += timedelta(days=3*30.5)  # Start a new period every 3 months
        start_date = start_date.replace(day=15)

    return periods

start_date = '2021-03-15'
num_periods = 7

periods = divide_periods(start_date, num_periods)
periods

[('2021-03-15', '2021-12-14'),
 ('2021-06-15', '2022-03-14'),
 ('2021-09-15', '2022-06-14'),
 ('2021-12-15', '2022-09-14'),
 ('2022-03-15', '2022-12-14'),
 ('2022-06-15', '2023-03-14'),
 ('2022-09-15', '2023-06-14')]

In [4]:
"""
dataset of desired period
"""

transaction = transaction[(transaction['created_at'] >= periods[PERIOD-1][0]) & (transaction['created_at'] <= periods[PERIOD-1][1])]
transaction

,account_id,code,quantity,price,side,created_at
0,8a9fa71f8b77775af51dfb5db302f272,290270,200,5960.0,bid,2021-03-15 08:40:18.983000+09:00
2,8a9fa71f8b77775af51dfb5db302f272,043200,3000,1480.0,bid,2021-03-15 08:41:05.676000+09:00
3,4e1334d8f5e21bb282aff7ee073efbcb,011785,17,91600.0,bid,2021-03-15 08:41:41.135000+09:00
4,8a9fa71f8b77775af51dfb5db302f272,262260,100,39550.0,bid,2021-03-15 08:41:41.807000+09:00
5,4e1334d8f5e21bb282aff7ee073efbcb,064260,501,9000.0,bid,2021-03-15 08:42:06.243000+09:00
...,...,...,...,...,...,...
178621,a74b4803cdd732be0b0461ddfcb83e7e,066570,10,134000.0,bid,2021-12-13 15:22:14.669000+09:00
178622,a678aa60304cc97bd35597e3a47af90d,192250,3891,2570.0,bid,2021-12-13 15:23:09.613000+09:00
178624,252892d72d54126b502cc9903d03fcd5,096530,78,70600.0,bid,2021-12-13 15:25:51.568000+09:00
178626,252892d72d54126b502cc9903d03fcd5,096530,78,70800.0,bid,2021-12-13 15:26:22.415000+09:00


# Filter user, item (order_processed_filtered.pkl)

## item

In [3]:
fdr.StockListing('KRX')

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,76600,3,0,0.00,76700,77100,76400,11304316,866836486100,457285343330000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,137500,1,1100,0.81,135800,137500,135800,1846781,252186160000,100100325187500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,416000,2,-3000,-0.72,416500,420500,414500,138131,57654205000,97344000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,756000,2,-14000,-1.82,765000,769000,753000,60943,46175385000,53807544000000,71174000,STK
4,005935,KR7005931001,삼성전자우,KOSPI,,61700,3,0,0.00,61500,61900,61200,714390,44011793156,50772109390000,822886700,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780,217320,KR7217320001,썬테크,KONEX,일반기업부,3120,0,0,0.00,0,0,0,0,0,3276000000,1050000,KNX
2781,288490,KR7288490006,나라소프트,KONEX,일반기업부,61,1,5,8.93,58,61,54,208285,11799602,3197298652,52414732,KNX
2782,245450,KR7245450002,씨앤에스링크,KONEX,일반기업부,1700,1,102,6.38,1500,1837,1500,82,133537,2685932000,1579960,KNX
2783,308700,KR7308700004,테크엔,KONEX,일반기업부,587,2,-2,-0.34,587,587,587,2,1174,2348000000,4000000,KNX


In [5]:
stocks = fdr.StockListing('KRX')
stocks_marcap = stocks.sort_values(by='Marcap', ascending=False)
stocks_marcap = stocks.iloc[:int(len(stocks)*0.95)]
transaction = transaction[transaction['code'].isin(stocks_marcap['Code'].unique())]
transaction

,account_id,code,quantity,price,side,created_at
0,8a9fa71f8b77775af51dfb5db302f272,290270,200,5960.0,bid,2021-03-15 08:40:18.983000+09:00
2,8a9fa71f8b77775af51dfb5db302f272,043200,3000,1480.0,bid,2021-03-15 08:41:05.676000+09:00
3,4e1334d8f5e21bb282aff7ee073efbcb,011785,17,91600.0,bid,2021-03-15 08:41:41.135000+09:00
4,8a9fa71f8b77775af51dfb5db302f272,262260,100,39550.0,bid,2021-03-15 08:41:41.807000+09:00
5,4e1334d8f5e21bb282aff7ee073efbcb,064260,501,9000.0,bid,2021-03-15 08:42:06.243000+09:00
...,...,...,...,...,...,...
178621,a74b4803cdd732be0b0461ddfcb83e7e,066570,10,134000.0,bid,2021-12-13 15:22:14.669000+09:00
178622,a678aa60304cc97bd35597e3a47af90d,192250,3891,2570.0,bid,2021-12-13 15:23:09.613000+09:00
178624,252892d72d54126b502cc9903d03fcd5,096530,78,70600.0,bid,2021-12-13 15:25:51.568000+09:00
178626,252892d72d54126b502cc9903d03fcd5,096530,78,70800.0,bid,2021-12-13 15:26:22.415000+09:00


In [6]:
DAYS = 30

transaction_stock_df = pd.read_csv('transaction_stock_df.csv', index_col='Date')
print('transaction_stock_df shape (before): ', transaction_stock_df.shape)
transaction_stock_df = transaction_stock_df.dropna(axis=1) 

# see if theres any consecutive same values in the dataframe
def is_consecutive_same(x):
    count = 0
    for i in range(1, len(x)):
        if x[i] == x[i-1]:
            count += 1
        else:
            count = 0
        if count >= DAYS:
            return True
    return False

same_columns = []
for column in tqdm(transaction_stock_df.columns):
    if is_consecutive_same(transaction_stock_df[column].values):
        same_columns.append(column)

# remove same_columns from transaction_stock_df
transaction_stock_df = transaction_stock_df.drop(columns=same_columns)

# print shape
print('transaction_stock_df shape (after): ', transaction_stock_df.shape)

transaction_stock_df shape (before):  (945, 2449)


100%|██████████| 2150/2150 [00:00<00:00, 3800.79it/s]

transaction_stock_df shape (after):  (945, 1983)


In [7]:
stocks_to_use = transaction_stock_df.columns
transaction = transaction[transaction['code'].isin(stocks_to_use)]
transaction

,account_id,code,quantity,price,side,created_at
0,8a9fa71f8b77775af51dfb5db302f272,290270,200,5960.0,bid,2021-03-15 08:40:18.983000+09:00
2,8a9fa71f8b77775af51dfb5db302f272,043200,3000,1480.0,bid,2021-03-15 08:41:05.676000+09:00
3,4e1334d8f5e21bb282aff7ee073efbcb,011785,17,91600.0,bid,2021-03-15 08:41:41.135000+09:00
5,4e1334d8f5e21bb282aff7ee073efbcb,064260,501,9000.0,bid,2021-03-15 08:42:06.243000+09:00
6,4e1334d8f5e21bb282aff7ee073efbcb,100790,55,8520.0,bid,2021-03-15 08:42:25.764000+09:00
...,...,...,...,...,...,...
178621,a74b4803cdd732be0b0461ddfcb83e7e,066570,10,134000.0,bid,2021-12-13 15:22:14.669000+09:00
178622,a678aa60304cc97bd35597e3a47af90d,192250,3891,2570.0,bid,2021-12-13 15:23:09.613000+09:00
178624,252892d72d54126b502cc9903d03fcd5,096530,78,70600.0,bid,2021-12-13 15:25:51.568000+09:00
178626,252892d72d54126b502cc9903d03fcd5,096530,78,70800.0,bid,2021-12-13 15:26:22.415000+09:00


## user

In [8]:
transaction['account_id'].value_counts()

account_id
7ced0b2f54382883270596d34b8e8c43    4962
b903738f6a00847a8162db3468e2d4e6    2808
81003180849e3a8d49fe2faf0dae553e    1952
b7916a99b93fbc2a69599a0be71f175e    1365
1d509e8ae762b093d133c3c8194e8aa9    1326
                                    ... 
c76574f7edfe0da17f9876f1b3113b05       1
b3b6b2bafa895d0c3e33b6fcf572ccec       1
5305cf5bc7a7a34d24fa6032e8f56e1a       1
2f3bb9393651214d4a25e531abbe53ba       1
a678aa60304cc97bd35597e3a47af90d       1
Name: count, Length: 4860, dtype: int64

In [9]:
print('num of user (before): ', len(transaction['account_id'].unique()))

# Assuming your DataFrame is named df and has a column 'user'
# Replace 'user' with the actual column name in your DataFrame

# Calculate the user frequencies
user_counts = transaction['account_id'].value_counts()

# Determine the threshold for the top 95% users
threshold = int(0.95 * len(user_counts))

# Get the users to keep
users_to_keep = user_counts.tail(threshold).index

# Filter the DataFrame to include only rows with users to be kept
transaction = transaction[transaction['account_id'].isin(users_to_keep)]

print('num of user (after): ', len(transaction['account_id'].unique()))

# Now, transaction contains the DataFrame with only the top 95% most frequent users
transaction

num of user (before):  4860
num of user (after):  4617


,account_id,code,quantity,price,side,created_at
3,4e1334d8f5e21bb282aff7ee073efbcb,011785,17,91600.0,bid,2021-03-15 08:41:41.135000+09:00
5,4e1334d8f5e21bb282aff7ee073efbcb,064260,501,9000.0,bid,2021-03-15 08:42:06.243000+09:00
6,4e1334d8f5e21bb282aff7ee073efbcb,100790,55,8520.0,bid,2021-03-15 08:42:25.764000+09:00
23,a8931fd6d104f8883974e2cce26004bf,303030,6666,3750.0,bid,2021-03-15 08:50:54.424000+09:00
25,a8931fd6d104f8883974e2cce26004bf,950110,2240,8370.0,bid,2021-03-15 08:51:15.851000+09:00
...,...,...,...,...,...,...
178621,a74b4803cdd732be0b0461ddfcb83e7e,066570,10,134000.0,bid,2021-12-13 15:22:14.669000+09:00
178622,a678aa60304cc97bd35597e3a47af90d,192250,3891,2570.0,bid,2021-12-13 15:23:09.613000+09:00
178624,252892d72d54126b502cc9903d03fcd5,096530,78,70600.0,bid,2021-12-13 15:25:51.568000+09:00
178626,252892d72d54126b502cc9903d03fcd5,096530,78,70800.0,bid,2021-12-13 15:26:22.415000+09:00


In [10]:
transaction['account_id'].value_counts()

account_id
60ca743c01ed2958327ec406805a2e78    97
7468ef5c6ce0aa4e416385a5f4f40724    97
8c9c1e53f584a8ef3da63e549667dc47    97
e1f8c2453350fcda40138cfc175b1a6c    97
bb9a010b0c771903dc926f32032e4700    96
                                    ..
b3b6b2bafa895d0c3e33b6fcf572ccec     1
5305cf5bc7a7a34d24fa6032e8f56e1a     1
2f3bb9393651214d4a25e531abbe53ba     1
27de963bb31869c150ec51c054a80243     1
a678aa60304cc97bd35597e3a47af90d     1
Name: count, Length: 4617, dtype: int64

In [57]:
# save transaction as pickle
transaction.to_pickle('order_processed_filtered.pkl')

# Add user portfolio (order_processed_filtered_portfolio.pkl)

In [58]:
# read transaction from pickle
transaction = pd.read_pickle('order_processed_filtered.pkl')

In [11]:

data_list = []

grouped = transaction.groupby('account_id')
for name, group in tqdm(grouped, total=len(grouped), desc="Processing groups"):
    # Assuming df is your DataFrame
    # Sort the DataFrame by 'created_at' to ensure correct order
    group = group.sort_values(by='created_at')

    # Initialize an empty dictionary to store cumulative portfolios
    cumulative_portfolios = {}

    # Function to update portfolio based on the row's information
    def update_portfolio(row):
        account_id = row['account_id']
        code = row['code']
        quantity = row['quantity']

        if account_id not in cumulative_portfolios:
            cumulative_portfolios[account_id] = {}

        if code not in cumulative_portfolios[account_id]:
            cumulative_portfolios[account_id][code] = 0

        if row['side'] == 'bid':
            cumulative_portfolios[account_id][code] += quantity
        elif row['side'] == 'ask':
            cumulative_portfolios[account_id][code] -= quantity

        # Remove keys with non-positive values
        cumulative_portfolios[account_id] = {k: v for k, v in cumulative_portfolios[account_id].items() if v > 0}

        # Assign the cumulative portfolio to the row
        row['portfolio'] = dict(cumulative_portfolios[account_id])
        return row

    # Apply the function to each row of the DataFrame
    group = group.apply(update_portfolio, axis=1)

    # Append the DataFrame to the list
    data_list.append(group)

# concat all data into a single dataframe
transaction = pd.concat(data_list, ignore_index=True)
transaction = transaction.sort_values(by='created_at', ignore_index=True)

Processing groups:   0%|          | 9/4617 [00:00<00:55, 83.29it/s]

Processing groups: 100%|██████████| 4617/4617 [00:39<00:00, 116.81it/s]


In [12]:


# remove rows where side is 'ask'
transaction = transaction[transaction['side'] == 'bid'].reset_index(drop=True)
# create a new clumn 'portfolio_len' which is the length of portfolio
transaction['portfolio_len'] = transaction['portfolio'].apply(lambda x: len(x))
# remove rows where portfolio_len is 0
transaction = transaction[transaction['portfolio_len'] != 0].reset_index(drop=True)
transaction

,account_id,code,quantity,price,side,created_at,portfolio,portfolio_len
0,4e1334d8f5e21bb282aff7ee073efbcb,011785,17,91600.0,bid,2021-03-15 08:41:41.135000+09:00,{'011785': 17},1
1,4e1334d8f5e21bb282aff7ee073efbcb,064260,501,9000.0,bid,2021-03-15 08:42:06.243000+09:00,"{'011785': 17, '064260': 501}",2
2,4e1334d8f5e21bb282aff7ee073efbcb,100790,55,8520.0,bid,2021-03-15 08:42:25.764000+09:00,"{'011785': 17, '064260': 501, '100790': 55}",3
3,a8931fd6d104f8883974e2cce26004bf,303030,6666,3750.0,bid,2021-03-15 08:50:54.424000+09:00,{'303030': 6666},1
4,a8931fd6d104f8883974e2cce26004bf,950110,2240,8370.0,bid,2021-03-15 08:51:15.851000+09:00,"{'303030': 6666, '950110': 2240}",2
...,...,...,...,...,...,...,...,...
38816,a74b4803cdd732be0b0461ddfcb83e7e,066570,10,134000.0,bid,2021-12-13 15:22:14.669000+09:00,{'066570': 10},1
38817,a678aa60304cc97bd35597e3a47af90d,192250,3891,2570.0,bid,2021-12-13 15:23:09.613000+09:00,{'192250': 3891},1
38818,252892d72d54126b502cc9903d03fcd5,096530,78,70600.0,bid,2021-12-13 15:25:51.568000+09:00,{'096530': 79},1
38819,252892d72d54126b502cc9903d03fcd5,096530,78,70800.0,bid,2021-12-13 15:26:22.415000+09:00,{'096530': 157},1


In [13]:
# get statistics of portfolio_len
transaction['portfolio_len'].describe()

count    38821.000000
mean         4.461297
std          4.639017
min          1.000000
25%          2.000000
50%          3.000000
75%          6.000000
max         47.000000
Name: portfolio_len, dtype: float64

In [14]:
"""
portfolio conversion: dict -> comma separated string
"""
transaction['portfolio'] = transaction['portfolio'].apply(lambda x: list(map(str, x.keys())))
# convert set to comma separated string
transaction['portfolio'] = transaction['portfolio'].apply(lambda x: ','.join(map(str, x)))
transaction

,account_id,code,quantity,price,side,created_at,portfolio,portfolio_len
0,4e1334d8f5e21bb282aff7ee073efbcb,011785,17,91600.0,bid,2021-03-15 08:41:41.135000+09:00,011785,1
1,4e1334d8f5e21bb282aff7ee073efbcb,064260,501,9000.0,bid,2021-03-15 08:42:06.243000+09:00,"011785,064260",2
2,4e1334d8f5e21bb282aff7ee073efbcb,100790,55,8520.0,bid,2021-03-15 08:42:25.764000+09:00,"011785,064260,100790",3
3,a8931fd6d104f8883974e2cce26004bf,303030,6666,3750.0,bid,2021-03-15 08:50:54.424000+09:00,303030,1
4,a8931fd6d104f8883974e2cce26004bf,950110,2240,8370.0,bid,2021-03-15 08:51:15.851000+09:00,"303030,950110",2
...,...,...,...,...,...,...,...,...
38816,a74b4803cdd732be0b0461ddfcb83e7e,066570,10,134000.0,bid,2021-12-13 15:22:14.669000+09:00,066570,1
38817,a678aa60304cc97bd35597e3a47af90d,192250,3891,2570.0,bid,2021-12-13 15:23:09.613000+09:00,192250,1
38818,252892d72d54126b502cc9903d03fcd5,096530,78,70600.0,bid,2021-12-13 15:25:51.568000+09:00,096530,1
38819,252892d72d54126b502cc9903d03fcd5,096530,78,70800.0,bid,2021-12-13 15:26:22.415000+09:00,096530,1


In [15]:
"""
see portfolio outliers
"""

# what is 90% quantile of portfolio_len? and corresponding number of data length when we filter by the quantile
print('90% quantile: ', transaction['portfolio_len'].quantile(0.9))
print('data length: ', len(transaction[transaction['portfolio_len'] <= transaction['portfolio_len'].quantile(0.9)]))
# what is ratio against total data length? make it into percentage and ceil it
print('data percentage: ', np.ceil(len(transaction[transaction['portfolio_len'] <= transaction['portfolio_len'].quantile(0.9)]) / len(transaction) * 100), '%')
print('')
# what is 95% quantile of portfolio_len? and corresponding number of data length when we filter by the quantile
print('95% quantile: ', transaction['portfolio_len'].quantile(0.95))
print('data length: ', len(transaction[transaction['portfolio_len'] <= transaction['portfolio_len'].quantile(0.95)]))
# what is ratio against total data length? make it into percentage and ceil it
print('data percentage: ', np.ceil(len(transaction[transaction['portfolio_len'] <= transaction['portfolio_len'].quantile(0.95)]) / len(transaction) * 100), '%')
print('')
# what is 99% quantile of portfolio_len? and corresponding number of data length when we filter by the quantile
print('99% quantile: ', transaction['portfolio_len'].quantile(0.99))
print('data length: ', len(transaction[transaction['portfolio_len'] <= transaction['portfolio_len'].quantile(0.99)]))
# what is ratio against total data length? make it into percentage and ceil it
print('data percentage: ', np.ceil(len(transaction[transaction['portfolio_len'] <= transaction['portfolio_len'].quantile(0.99)]) / len(transaction) * 100), '%')
print('')

90% quantile:  9.0
data length:  34962
data percentage:  91.0 %

95% quantile:  13.0
data length:  37153
data percentage:  96.0 %

99% quantile:  24.0
data length:  38474
data percentage:  100.0 %



In [16]:
"""
if you want to remove outliers
"""

# transaction = transaction[transaction['portfolio_len'] <= transaction['portfolio_len'].quantile(0.95)]
# transaction = transaction.reset_index(drop=True)
# transaction = transaction.drop(columns=['portfolio_len'])
# print('transaction shape: ', transaction.shape)

'\nif you want to remove outliers\n'

In [65]:
# save transaction_portfolio_processed as pickle
transaction.to_pickle('order_processed_filtered_portfolio.pkl')

# Add edge features (order_processed_filtered_portfolio_features.pkl)

In [66]:
# read flie
transaction = pd.read_pickle('order_processed_filtered_portfolio.pkl')

In [17]:


items_1 = transaction['code'].unique()
items_2 = transaction_stock_df.columns

# see if items_2 contains all items_1
print('items_1 is subset of items_2: ', set(items_1).issubset(set(items_2)))

items_1 is subset of items_2:  True


In [18]:
"""
create edge features col 
"""



def find_feature(code, date, transaction_stock_df):
    date = str(date)
    date = datetime.strptime(date[:10], '%Y-%m-%d').date() # e.g., 2021-03-15
    # retrieve past 30 days of prices from transaction_stock_df
    curr_date = transaction_stock_df.index[transaction_stock_df.index.get_loc(str(date)) - 1] # e.g., 2021-03-12 
    prev_date = transaction_stock_df.index[transaction_stock_df.index.get_loc(curr_date) - DAYS]
    transaction_code = transaction_stock_df[code][prev_date:curr_date].values.tolist()
    return transaction_code

transaction['created_at'] = pd.to_datetime(transaction['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S.%f')

transaction['edge_features'] = transaction.apply(lambda x: find_feature(x['code'], x['created_at'], transaction_stock_df), axis=1)

transaction

,account_id,code,quantity,price,side,created_at,portfolio,portfolio_len,edge_features
0,4e1334d8f5e21bb282aff7ee073efbcb,011785,17,91600.0,bid,2021-03-15 08:41:41.135000,011785,1,"[112500, 109000, 122500, 101000, 98800, 104000..."
1,4e1334d8f5e21bb282aff7ee073efbcb,064260,501,9000.0,bid,2021-03-15 08:42:06.243000,"011785,064260",2,"[4595, 4520, 4515, 4340, 4425, 4435, 4480, 444..."
2,4e1334d8f5e21bb282aff7ee073efbcb,100790,55,8520.0,bid,2021-03-15 08:42:25.764000,"011785,064260,100790",3,"[5834, 5748, 5691, 5481, 5767, 5767, 5796, 574..."
3,a8931fd6d104f8883974e2cce26004bf,303030,6666,3750.0,bid,2021-03-15 08:50:54.424000,303030,1,"[3175, 3150, 3100, 3005, 3200, 3255, 3260, 329..."
4,a8931fd6d104f8883974e2cce26004bf,950110,2240,8370.0,bid,2021-03-15 08:51:15.851000,"303030,950110",2,"[7950, 7960, 7840, 7750, 7920, 7830, 7870, 786..."
...,...,...,...,...,...,...,...,...,...
38816,a74b4803cdd732be0b0461ddfcb83e7e,066570,10,134000.0,bid,2021-12-13 15:22:14.669000,066570,1,"[120500, 120500, 123000, 122000, 122500, 12300..."
38817,a678aa60304cc97bd35597e3a47af90d,192250,3891,2570.0,bid,2021-12-13 15:23:09.613000,192250,1,"[2305, 2500, 2565, 2555, 2455, 2445, 2555, 274..."
38818,252892d72d54126b502cc9903d03fcd5,096530,78,70600.0,bid,2021-12-13 15:25:51.568000,096530,1,"[53200, 55100, 55700, 54900, 53700, 53200, 507..."
38819,252892d72d54126b502cc9903d03fcd5,096530,78,70800.0,bid,2021-12-13 15:26:22.415000,096530,1,"[53200, 55100, 55700, 54900, 53700, 53200, 507..."


In [69]:
# save transaction as pickle
transaction.to_pickle('order_processed_filtered_portfolio_features.pkl')

# 1st preprocessing

In [70]:
# read file
transaction = pd.read_pickle('order_processed_filtered_portfolio_features.pkl')

In [19]:
# change col name: account_id to user_id, and code to item_id
transaction = transaction.rename(columns={'account_id': 'user_id', 'code': 'item_id'})

"""
timestamp
"""

# str -> int
# for example, '2021-03-15 08:40:18.983000+09:00' -> 20210315084018983
def convert_to_int(datetime_str):
    datetime_str = str(datetime_str)
    numeric_str = ''.join(filter(str.isdigit, datetime_str))
    return int(numeric_str[:-7]) # 일 단위: 8

# Assuming df is your DataFrame and 'datetime_column' is the column with datetime strings
transaction['created_at'] = transaction['created_at'].apply(lambda x: convert_to_int(x))


"""
user id, item id, ts id
"""

# map user_id starting from 0
map_user_id = {user_id: i for i, user_id in enumerate(transaction['user_id'].unique())}
transaction['user_id'] = transaction['user_id'].map(map_user_id)

# map item_id starting from 0
map_item_id = {item_id: i for i, item_id in enumerate(transaction['item_id'].unique())}
transaction['item_id'] = transaction['item_id'].map(map_item_id)

# # convert 'ts', starting from 0
# asis = transaction['created_at'] 
# transaction['created_at'] = pd.to_numeric(transaction['created_at'], errors='coerce')
# transaction['created_at'] = transaction['created_at'] - transaction['created_at'].min()
# tobe = transaction['created_at']
# map_ts_id = {a: t for a, t in zip(asis, tobe)}

# save user_id_map, item_id_map, ts_id_map as pkl
with open(save_path+'map_user_id.pkl', 'wb') as f:
    pickle.dump(map_user_id, f)
with open(save_path+'map_item_id.pkl', 'wb') as f:
    pickle.dump(map_item_id, f)
# with open('map_ts_id.pkl', 'wb') as f:
#     pickle.dump(map_ts_id, f)

transaction

,user_id,item_id,quantity,price,side,created_at,portfolio,portfolio_len,edge_features
0,0,0,17,91600.0,bid,2021031508414,011785,1,"[112500, 109000, 122500, 101000, 98800, 104000..."
1,0,1,501,9000.0,bid,2021031508420,"011785,064260",2,"[4595, 4520, 4515, 4340, 4425, 4435, 4480, 444..."
2,0,2,55,8520.0,bid,2021031508422,"011785,064260,100790",3,"[5834, 5748, 5691, 5481, 5767, 5767, 5796, 574..."
3,1,3,6666,3750.0,bid,2021031508505,303030,1,"[3175, 3150, 3100, 3005, 3200, 3255, 3260, 329..."
4,1,4,2240,8370.0,bid,2021031508511,"303030,950110",2,"[7950, 7960, 7840, 7750, 7920, 7830, 7870, 786..."
...,...,...,...,...,...,...,...,...,...
38816,4615,136,10,134000.0,bid,2021121315221,066570,1,"[120500, 120500, 123000, 122000, 122500, 12300..."
38817,4616,476,3891,2570.0,bid,2021121315230,192250,1,"[2305, 2500, 2565, 2555, 2455, 2445, 2555, 274..."
38818,4435,354,78,70600.0,bid,2021121315255,096530,1,"[53200, 55100, 55700, 54900, 53700, 53200, 507..."
38819,4435,354,78,70800.0,bid,2021121315262,096530,1,"[53200, 55100, 55700, 54900, 53700, 53200, 507..."


## store data

In [72]:
"""
transaction.npy
edge features 
"""
transaction_features = np.array(transaction['edge_features'].values.tolist())
np.save(save_path+'transaction.npy', transaction_features)

In [73]:
"""
transaction.csv
used in preprocess_data.py
"""

# change col name: created_at to timestamp, side to state_label, quantity to features
transaction = transaction.rename(columns={'created_at': 'timestamp', 'side': 'state_label', 'quantity': 'features'})
transction = transaction[['user_id', 'item_id', 'timestamp', 'state_label', 'features', 'portfolio']]
transction.to_csv(save_path+'transaction.csv', index=False)

# 2nd preprocessing

In [ ]:
"""
terminal run to 2nd preprocessing

 ml_transaction.csv
 ml_transaction_node.npy
 ml_transaction.npy
"""

# python utils/preprocess_data.py --bipartite --period all

In [74]:
"""
check the resulted data
"""

import pandas as pd
import numpy as np
save_path = f'period_{PERIOD}/'

# interactions
ml_transaction = pd.read_json(save_path+'ml_transaction.json', orient='records')
print('ml_transaction: ', ml_transaction.shape)

# node features
ml_transaction_node = np.load(save_path+'ml_transaction_node.npy')
print('ml_transaction_node: ', ml_transaction_node.shape)

# inter features
ml_transaction_feat = np.load(save_path+'ml_transaction.npy')
print('ml_transaction_feat: ', ml_transaction_feat.shape)

ml_transaction:  (79352, 6)
ml_transaction_node:  (10117, 172)
ml_transaction_feat:  (79353, 31)
